<a href="https://colab.research.google.com/github/urvashiverma2608/Projectsecure/blob/master/Assignment2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importing libraries as necessary 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import os
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,KFold
import keras

In [3]:
# convert dataframe for the test data 
dir = '/content/test_signal.csv'
messages = pd.read_csv(os.path.join(dir), sep=",")
messages.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,C00000,-0.169,-0.174,-0.184,-0.189,-0.200,-0.210,-0.221,-0.226,-0.226,...,0.375,0.386,0.386,0.360,0.323,0.282,0.240,0.184,0.132,0.090
1,C00001,-0.184,-0.174,-0.169,-0.164,-0.158,-0.158,-0.158,-0.153,-0.153,...,-0.086,-0.117,-0.143,-0.148,-0.153,-0.153,-0.153,-0.153,-0.153,-0.158
2,C00002,1.050,1.622,2.143,2.552,2.653,2.675,2.614,2.249,1.734,...,-1.342,-1.354,-1.365,-1.370,-1.376,-1.370,-1.365,-1.354,-1.342,-1.326
3,C00003,2.259,2.667,3.046,3.376,3.583,3.579,3.334,2.872,2.514,...,-0.119,-0.119,-0.119,-0.116,-0.114,-0.109,-0.104,-0.097,-0.090,-0.083
4,C00004,-0.203,-0.203,-0.201,-0.201,-0.201,-0.199,-0.194,-0.189,-0.185,...,-0.008,0.020,0.039,0.044,0.046,0.046,0.049,0.049,0.051,0.051


In [4]:
# converting train data into data frame to read it better
dir = '/content/train_signal.csv'
messages1 = pd.read_csv(os.path.join(dir), sep=",")
messages1.head()

<ipython-input-4-8fc350230a7b>:3: DtypeWarning: Columns (4141) have mixed types. Specify dtype option on import or set low_memory=False.
  messages1 = pd.read_csv(os.path.join(dir), sep=",")


,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,N,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,N,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,N,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
3,B00003,~,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,~,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207


In [5]:
# recognising missing values in the training data
x= messages1.iloc[:,:-1].values
x

array([['B00000', 'N', -0.107, ..., -0.804, -0.724, -0.63],
       ['B00001', 'N', 2.762, ..., 0.368, 0.397, 0.426],
       ['B00002', 'N', -0.246, ..., 0.194, 0.159, 0.119],
       ...,
       ['B12505', 'O', -0.39, ..., -0.457, -0.447, -0.436],
       ['B12506', 'N', 3.147, ..., -0.042, -0.008, 0.034],
       ['B12507', 'N', -0.665, ..., nan, nan, nan]], dtype=object)

In [6]:
# filling missing values using the function
imputer= SimpleImputer(missing_values ='NaN', strategy='mean')  
imputer

SimpleImputer(missing_values='NaN')

In [7]:
#picking out the main feature in the data 
y = messages1.ID
X = messages1.drop('ID', axis=1)
# split the data into test and train 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

     Type     X0     X1     X2     X3     X4     X5     X6     X7     X8  ...  \
1655    N  0.081  0.081  0.036 -0.078 -0.197  0.050  0.542  1.150  1.819  ...   
7411    N -0.165 -0.128 -0.084 -0.055 -0.018  0.004  0.026  0.048  0.070  ...   
6541    N -0.591 -0.560 -0.530 -0.499 -0.446 -0.415 -0.446 -0.522 -0.560  ...   
2277    O -0.396 -0.517 -0.558 -0.579 -0.591 -0.604 -0.612 -0.624 -0.633  ...   
5047    N  0.550  0.730  0.967  1.180  1.328  1.450  1.549  1.614  1.688  ...   

      X5990  X5991  X5992  X5993  X5994  X5995  X5996  X5997  X5998  X5999  
1655  0.233  0.111 -0.064 -0.408 -0.775 -1.113 -1.319 -1.338 -1.208 -0.858  
7411  0.026  0.040  0.055  0.062  0.062  0.062  0.062  0.062  0.070  0.070  
6541  0.359  0.366  0.374  0.381  0.381  0.374  0.351  0.328  0.297  0.267  
2277 -0.396 -0.396 -0.396 -0.400 -0.400 -0.396 -0.392 -0.388 -0.388 -0.384  
5047 -0.384 -0.384 -0.384 -0.376 -0.376 -0.376 -0.384 -0.384 -0.384 -0.392  

[5 rows x 6001 columns]
(10006, 6001)



In [8]:
disp = confusion_matrix(X_test, y_test)
disp
print(classification_report(y_test, y_test_pred))
rfc = GridSearchCV(logreg, param_grid, cv = 5)

TypeError: ignored

In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_auc_and_roc(y_true, scores, col='red'):
    fpr, tpr, _ = roc_curve(y_true, scores)
    calc_auc = auc(fpr, tpr)
        
    plt.title("Recieving Operating Characteristic")
    plt.plot(fpr, tpr, col, label="AUC %0.3f" % calc_auc)
    plt.plot([0, 1], [0, 1], "b--")
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.legend(loc="lower right")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [10]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential([
    Dense(32, activation='relu', input_shape=(6000,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                192032    
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 193,121
Trainable params: 193,121
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])
X_train=np.asarray(X_train).astype(np.float32)
y_train=np.asarray(y_train).astype(np.float32)
model.compile()
hist = model.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_train, y_train))

ValueError: ignored

In [16]:
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = np.array(y_train).reshape(y_train.shape[0], 1)
y_test = np.array(y_test).reshape(y_test.shape[0], 1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test)

(10006, 6001, 1)
(2502, 6001, 1)
(10006, 1)
(2502, 1)
[['B07563']
 ['B09168']
 ['B09465']
 ...
 ['B11181']
 ['B03374']
 ['B11730']]


In [17]:
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout , Conv2D

In [18]:
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation='relu', input_shape = (6000,)))
model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation='relu')) 
model.add(Conv1D(filters=128, kernel_size=(5,), padding='same', activation='relu'))    

model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = 1024, activation='relu'))

model.add(Dense(units = 1, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

ValueError: ignored

In [19]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

NameError: ignored

In [ ]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical

y_train=keras.utils.np_utils.to_categorical(y_train)
y_test=keras.utils.np_utils.to_categorical(y_test)

history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50 , batch_size=32)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50 , batch_size=32)

ValueError: ignored